Regression:


Models:

Unem = $\beta_0 + \beta_1 Agri + \beta_2 Mini + \beta_3 Util + \beta_4 EFW + \beta_5 M4 + \beta_6 M4I + \epsilon$ ..................................(1)

Unem = $\beta_0 + \beta_1 Agri + \beta_2 Mini + \beta_3 Util + \beta_5 M4 + \beta_6 M4I + \epsilon$ ..............................................(2)

Results:

#### Rates

##### Model 1:

$Unem = 45.7968 - 0.0091 Agri - 0.2354 Mini - 0.0808 Util - 4.7977 EFW - 16.3228 M4 - 1.0745 M4I $

Insig: Agri, M4I

##### Model 2:

$Unem = 7.4812 - 0.0188 Agri - 0.2428 Mini - 0.1145 Util - 18.8046 M4 -51.5287 M4I $

Insig: Agri 

#### Differenced Rates

##### Model 1:

$Unem = 0.0189 Agri - 0.0520 Mini + 0.0251 Util - 6.1323 EFW - 7.3195 M4 - 25.7390 M4I $

##### Model 2

$Unem = 0.0141 Agri - 0.0454 Mini - 0.0210 Util - 13.1908 M4 - 41.0566 M4I $

Insig: Utilities

$H_1$ : There is a positive correlation between money supply and production economy

$H_2$ : An increase in interest rates results in a decrease in the production

$H_3$ : Increase in money supply results in decrease in unemployment rates

$H_4$ : There exists a positive correlation between interest rates and unemployment rates

$H_5$ : The higher the economic freedom, the lower the unemployment rates and the higher the nominal GDP

### DAGs:

#### Rates with EFW

$H_1$: Fail to reject

$H_2$: Reject, fail to reject for mining

$H_3$: Fail to reject

$H_4$: Reject 

$H_5$ : Reject, fail to reject for an indirect relationship with utilities


#### Rates without EFW:

$H_1$: Fail to reject

$H_2$: Reject

$H_3$: Fail to reject

$H_4$: Reject 

#### Differenced Rates with EFW

$H_1$: Fail to reject for (Utilities and Mining), Reject for Agric

$H_2$: Reject

$H_3$: Fail to reject

$H_4$: Reject

$H_5$: Fail to reject, reject for Agric

#### Differenced Rates without EFW

$H_1$: Fail to reject, reject for Agric

$H_2$: Reject, fail to reject for Agric

$H_3$: Fail to reject

$H_4$: Reject
